# Polynomial Regression

## Importing the libraries

In [1]:
from glob import glob
import cv2
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, auc
from google.colab import drive
# import glob



from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing the dataset

In [ ]:
images_path = {}
images_path["malignant"] = glob("/content/drive/MyDrive/dani/GasHisSDB/160/Abnormal/*.png")
images_path["benign"] = glob("/content/drive/MyDrive/dani/GasHisSDB/160/Normal/*.png")


images_class = {
    'benign' : 0,
    'malignant' : 1
}

import sys

from tensorflow.keras.applications import ResNet101
from keras.models import Model
import keras

pre_trained_models = {}


from tensorflow.keras.applications import InceptionV3
import numpy as np

pre_trained_models["ResNet101"] = ResNet101(include_top=False, input_shape=(224, 224, 3), pooling="avg", weights = 'imagenet')
for layer in pre_trained_models["ResNet101"].layers:
    layer.trainable = False


# print(images_path["malignant"])
counter = 0




Y = []
Z = []
pos = 0
for label in images_path:
    counter = 0
    for image_path in images_path[label]:
        # print(counter, ' / ', all)
        counter += 1
        if(1):
          image = cv2.imread(image_path)
          image = cv2.resize(image,(224, 224))
          Z.append(image)
          
          if(counter % 10 == 0):
            temp = np.array(Z)
            

            feature_ResNet101 = pre_trained_models["ResNet101"].predict(temp)
            # print(feature_ResNet101)

            for b in feature_ResNet101:
              c = b.tolist()
              if(pos == 0):
                  X =  [c]
              else:
                X.append(c)


              pos += 1
              # print("shapeX:")
              # o = np.array(X)
              # print(o.shape)


           
            
            # print(X)
            Z = []
          

          Y.append(images_class[label])
          if(counter % 10000  == 0):
            print("counter:")
            print(counter)
            print("label:" + label)
        else:
          continue

print(np.size(X))
print(np.size(Y))







sample_list = X
new_array = np.array(sample_list)

# Displaying the array

file = open("X.txt", "w+")

# Saving the array in a text file
content = str(new_array)
file.write(content)
file.close()



sample_list = Y
new_array = np.array(sample_list)

# Displaying the array

file = open("Y.txt", "w+")

# Saving the array in a text file
content = str(new_array)
file.write(content)
file.close()

171458560/171446536 [==============================] - 1s 0us/step
counter:
10000
label:malignant
counter:
10000
label:benign
counter:
20000
label:benign
68165632
33284


## Training the Linear Regression model on the whole dataset

In [ ]:
x = np.array(X)
y = np.array(Y)

In [ ]:
from tempfile import TemporaryFile
outfile = TemporaryFile()
np.save(outfile, x)

np.savetxt('X_new.txt', x, fmt='%s')
np.savetxt('Y_new.txt', y, fmt='%s')


In [ ]:
Filename = "res101"


addressX = "/content/drive/MyDrive/dani/" + Filename + "_X.txt"
addressY = "/content/drive/MyDrive/dani/" + Filename + "_Y.txt"



np.savetxt(addressX, x, fmt='%s')
np.savetxt(addressY, x, fmt='%s')



In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
print(x.shape)
print(y.shape)

In [ ]:
from catboost import CatBoostClassifier
# Initialize data

maxXG_iHolder = 33
maxCAT_iHolder = 49
maxLGBM_iHolder = 10



learnXG_iHolder = 0.4
learnCAT_iHolder = 0.4
learnLGBM_iHolder = 0.3
maxXG = 0.9130783096334869
maxCAT = 0.9204886841578209
maxLGBM = 0.9154816743440817


for i in range(7,11):

  tempL = i / 10

  print("tempL:")
  print(tempL)

  X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxXG_iHolder)


  result = {}
  Classifier = XGBClassifier(learning_rate=tempL, n_estimators=150, random_state=0, seed=0, gamma=0)
  Classifier.fit(X_train, Y_train)
  Y_pred = Classifier.predict(X_test)
  ac = accuracy_score(Y_test, Y_pred)
  print("Accuracy score : ", ac)

  if(ac > maxXG):
    maxXG = ac
    learnXG_iHolder = i / 10


  X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxCAT_iHolder)
  

  model = CatBoostClassifier(iterations=1000,learning_rate=tempL,depth=2)
  model.fit(X_train, Y_train,verbose=False)
  Y_pred = model.predict(X_test)
  ac = accuracy_score(Y_test, Y_pred)
  print("Accuracy score : ", ac)

  if(ac > maxCAT):
    maxCAT = ac
    learnCAT_iHolder = i / 10


  X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxLGBM_iHolder)


  import lightgbm as lgb
  model = lgb.LGBMClassifier(learning_rate=tempL,max_depth=-5,random_state=0)
  model.fit(X_train, Y_train)
  Y_pred = model.predict(X_test)
  ac = accuracy_score(Y_test, Y_pred)
  print("Accuracy score : ", ac)

  if(ac > maxLGBM):
    maxLGBM = ac
    learnLGBM_iHolder = i / 10


  print("maxXG:")
  print(maxXG)

  print("maxCAT:")
  print(maxCAT)

  print("maxLGBM:")
  print(maxLGBM)





  
  print("learnXG_iHolder:")
  print(learnXG_iHolder)

  print("learnCAT_iHolder:")
  print(learnCAT_iHolder)

  print("learnLGBM_iHolder:")
  print(learnLGBM_iHolder)


  
  print("maxXG:")
  print(maxXG)

  print("maxCAT:")
  print(maxCAT)

  print("maxLGBM:")
  print(maxLGBM)


tempL:
0.7
Accuracy score :  0.9020628880432605
Accuracy score :  0.9152813939515322
Accuracy score :  0.9132785900260364
maxXG:
0.9130783096334869
maxCAT:
0.9204886841578209
maxLGBM:
0.9154816743440817
learnXG_iHolder:
0.4
learnCAT_iHolder:
0.4
learnLGBM_iHolder:
0.3
maxXG:
0.9130783096334869
maxCAT:
0.9204886841578209
maxLGBM:
0.9154816743440817
tempL:
0.8
Accuracy score :  0.9025635890246345
Accuracy score :  0.9125776086521129
Accuracy score :  0.8422791908672141
maxXG:
0.9130783096334869
maxCAT:
0.9204886841578209
maxLGBM:
0.9154816743440817
learnXG_iHolder:
0.4
learnCAT_iHolder:
0.4
learnLGBM_iHolder:
0.3
maxXG:
0.9130783096334869
maxCAT:
0.9204886841578209
maxLGBM:
0.9154816743440817
tempL:
0.9
Accuracy score :  0.905567794912878
Accuracy score :  0.9107750851191668
Accuracy score :  0.8442819947927098
maxXG:
0.9130783096334869
maxCAT:
0.9204886841578209
maxLGBM:
0.9154816743440817
learnXG_iHolder:
0.4
learnCAT_iHolder:
0.4
learnLGBM_iHolder:
0.3
maxXG:
0.9130783096334869
maxCAT

## Tube

In [2]:
Filename = "res101"


addressX = "/content/drive/MyDrive/dani/" + Filename + "_X.txt"
addressY = "/content/drive/MyDrive/dani/" + "main" + "_Y.txt"





b = np.loadtxt(addressX)
X = b

b = np.loadtxt(addressY)
Y = b

x = np.array(X)
y = np.array(Y)


print(b.shape)
print(x.shape)



(33284,)
(33284, 2048)


In [3]:
maxXG_iHolder = 33
maxCAT_iHolder = 49
maxLGBM_iHolder = 10

maxXG = 0
learnXG_iHolder = 0.4
learnCAT_iHolder = 0.4
learnLGBM_iHolder = 0.3



X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=maxCAT_iHolder)

In [4]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.3 MB/s 


In [ ]:
maxXG = 0
learnXG_iHolder = 0.4
learnCAT_iHolder = 0.4
learnLGBM_iHolder = 0.3

skipper = True

for tempGamma in range(0,51):
  tempGamma = tempGamma / 10
  for min_child_weight in range(1,11):
    for subsample in range(1,11):
      subsample = subsample / 10
      for colsample_bytree in range(1,11):
        colsample_bytree = colsample_bytree / 10
        for max_depth in range(1,11):
          for n_entimatore in range(50,401,50):

            if(skipper):
              result = {}
              Classifier = XGBClassifier(learning_rate=learnXG_iHolder,min_child_weight=min_child_weight,max_depth=max_depth, n_estimators=n_entimatore, subsample=subsample, colsample_bytree=colsample_bytree, gamma=tempGamma)
              Classifier.fit(X_train, Y_train)
              Y_pred = Classifier.predict(X_test)
              ac = accuracy_score(Y_test, Y_pred)
              print("Accuracy score : ", ac)
              print("params:")
              print("gamma:",tempGamma)
              print("subsample:",subsample)
              print("colsample_bytree:",colsample_bytree)
              print("max_depth:",max_depth)
              print("min_child_weight:",min_child_weight)
              print("n_entimatore:",n_entimatore)




              print()
              print("Best_params:")
              print("Best_gamma:",Best_gamma)
              print("Best_subsample:",Best_subsample)
              print("Best_colsample_bytree:",Best_colsample_bytree)
              print("Best_max_depth:",Best_max_depth)
              print("Best_min_child_weight:",Best_min_child_weight)
              print("Best_n_entimatore:",Best_n_entimatore)
              print("Best_ac:",maxXG)


              if(ac > maxXG):
                maxXG = ac
                Best_gamma = tempGamma
                Best_subsample = subsample
                Best_colsample_bytree = colsample_bytree
                Best_max_depth = max_depth
                Best_min_child_weight = min_child_weight
                Best_n_entimatore = n_entimatore


                print("Best_params:")
                print("Best_gamma:",Best_gamma)
                print("Best_subsample:",Best_subsample)
                print("Best_colsample_bytree:",Best_colsample_bytree)
                print("Best_max_depth:",Best_max_depth)
                print("Best_min_child_weight:",Best_min_child_weight)
                print("Best_n_entimatore:",Best_n_entimatore)
                    








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Streaming output truncated to the last 5000 lines.
max_depth: 3
min_child_weight: 1
n_entimatore: 300

Best_params:
Best_gamma: 0.0
Best_subsample: 0.1
Best_colsample_bytree: 0.2
Best_max_depth: 1
Best_min_child_weight: 1
Best_n_entimatore: 350
Best_ac: 0.87903064290006
Accuracy score :  0.8493891448027238
params:
gamma: 0.0
subsample: 0.1
colsample_bytree: 0.2
max_depth: 3
min_child_weight: 1
n_entimatore: 350

Best_params:
Best_gamma: 0.0
Best_subsample: 0.1
Best_colsample_bytree: 0.2
Best_max_depth: 1
Best_min_child_weight: 1
Best_n_entimatore: 350
Best_ac: 0.87903064290006
Accuracy score :  0.8527939114760665
params:
gamma: 0.0
subsample: 0.1
colsample_bytree: 0.2
max_depth: 3
min_child_weight: 1
n_entimatore: 400

Best_params:
Best_gamma: 0.0
Best_subsample: 0.1
Best_colsample_bytree: 0.2
Best_max_depth: 1
Best_min_child_weight: 1
Best_n_entimatore: 350
Best_ac: 0.87903064290006
Accuracy score :  0.8612056879631484
params:
gamma: 0.0
subsample: 0.1
colsample_bytree: 0.2
max_depth:

## Gamma Test

In [ ]:
maxXG = 0
learnXG_iHolder = 0.4
learnCAT_iHolder = 0.4
learnLGBM_iHolder = 0.3

In [ ]:
maxXG = 0
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]


Best_gamma = 0
Best_subsample = 0
Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0
Best_n_entimatore = 0



for gamma in GammaArr:
  subsample = 0
  min_child_weight = 0
  colsample_bytree = 0
  max_depth = 0
  n_entimatore = 150

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder, gamma=gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",gamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = gamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.9118766272781894
params:
gamma: 0.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 150

Best_params:
Best_gamma: 0
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 150
Best_ac: 0.9118766272781894
Accuracy score :  0.9118766272781894
params:
gamma: 1
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 150

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 150
Best_ac: 0.9118766272781894
Accuracy score :  0.9115762066893651
params:
gamma: 1.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 150

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_chi

In [ ]:
maxXG = 0
skipper = False
skipper = True

gamma = 0.5
GammaArr = [0.5,1,1.5,2,5,10]


Best_gamma = 0.5
Best_subsample = 0
Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0
Best_n_entimatore = 0



for n_entimatore in range(0,601,50):
  subsample = 0
  min_child_weight = 0
  colsample_bytree = 0
  max_depth = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",gamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = gamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.6092529541357901
params:
gamma: 0.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 0

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0.6092529541357901
Accuracy score :  0.900460644902864
params:
gamma: 0.5
subsample: 0
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 50

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 0
Best_ac: 0.6092529541357901
Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 50
Best_ac: 0.900460644902864
Accuracy score :  0.9070698978569998
params:
gamma: 0.5
subsample: 0
colsample_bytr

In [ ]:
maxXG = 0
skipper = False
skipper = True

Best_gamma = 0.5
tempGamma = 0.5
GammaArr = [0.5,1,1.5,2,5,10]
n_entimatore = 550
Best_n_entimatore = 550

Best_subsample = 0
Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0


for subsample in range(1,11):
  subsample = subsample / 10
  min_child_weight = 0
  colsample_bytree = 0
  max_depth = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.8518926497095934
params:
gamma: 0.5
subsample: 0.1
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 550

Best_params:
Best_gamma: 0.5
Best_subsample: 0
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 0.1
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.8518926497095934
Accuracy score :  0.8842379331063489
params:
gamma: 0.5
subsample: 0.2
colsample_bytree: 0
max_depth: 0
min_child_weight: 0
n_entimatore: 550

Best_params:
Best_gamma: 0.5
Best_subsample: 0.1
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.8518926497095934
Best_params:
Best_gamma: 0.5
Best_subsample: 0.2
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.8842379331063489
Accuracy score :  0.8997596635289405
params:
gamma: 0.5
subsa

## COLsample

In [ ]:
maxXG = 0
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]

Best_gamma = 0.5
tempGamma = 0.5



n_entimatore = 550
Best_n_entimatore = 550


Best_subsample = 1
subsample = 1


Best_colsample_bytree = 0
Best_max_depth = 0
Best_min_child_weight = 0


for colsample_bytree in range(1,11):
  colsample_bytree = colsample_bytree / 10
  min_child_weight = 0
  max_depth = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,colsample_bytree=colsample_bytree,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.9192870018025235
params:
gamma: 0.5
subsample: 1
colsample_bytree: 0.1
max_depth: 0
min_child_weight: 0
n_entimatore: 550

Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0
Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0.1
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9192870018025235
Accuracy score :  0.9207891047466453
params:
gamma: 0.5
subsample: 1
colsample_bytree: 0.2
max_depth: 0
min_child_weight: 0
n_entimatore: 550

Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0.1
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9192870018025235
Best_params:
Best_gamma: 0.5
Best_subsample: 1
Best_colsample_bytree: 0.2
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9207891047466453
Accuracy score :  0.9209893851391949
params:
gamma: 0.5
subsa

## Depth

In [ ]:
maxXG = 0.9191868616062487
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]

Best_gamma = 1
tempGamma = 1



n_entimatore = 550
Best_n_entimatore = 550


Best_subsample = 0.8
subsample = 0.8


Best_colsample_bytree = 0.8
colsample_bytree = 0.8


Best_max_depth = 0
Best_min_child_weight = 0


for max_depth in range(9,11):
  min_child_weight = 0

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,max_depth=max_depth,colsample_bytree=Best_colsample_bytree,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.9067694772681755
params:
gamma: 1
subsample: 0.8
colsample_bytree: 0.8
max_depth: 9
min_child_weight: 0
n_entimatore: 550

Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 0.8
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9191868616062487
Accuracy score :  0.906969757660725
params:
gamma: 1
subsample: 0.8
colsample_bytree: 0.8
max_depth: 10
min_child_weight: 0
n_entimatore: 550

Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 0.8
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9191868616062487
Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 0.8
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0.9191868616062487


## Min_child_weight

In [ ]:
maxXG = 0
skipper = False
skipper = True

GammaArr = [0.5,1,1.5,2,5,10]

Best_gamma = 1
tempGamma = 1



n_entimatore = 550
Best_n_entimatore = 550


Best_subsample = 0.8
subsample = 0.8

Best_colsample_bytree = 0.8
colsample_bytree = 0.8

# Best_max_depth = 5
Best_min_child_weight = 0


for min_child_weight in range(1,11):

  if(skipper):
    result = {}
    Classifier = XGBClassifier(learning_rate=learnXG_iHolder,min_child_weight=min_child_weight,max_depth=Best_max_depth,colsample_bytree=Best_colsample_bytree,subsample=subsample, gamma=Best_gamma, n_estimators=n_entimatore, random_state=0, seed=0)
    Classifier.fit(X_train, Y_train)
    Y_pred = Classifier.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)
    print("params:")
    print("gamma:",tempGamma)
    print("subsample:",subsample)
    print("colsample_bytree:",colsample_bytree)
    print("max_depth:",max_depth)
    print("min_child_weight:",min_child_weight)
    print("n_entimatore:",n_entimatore)



    print()
    print("Best_params:")
    print("Best_gamma:",Best_gamma)
    print("Best_subsample:",Best_subsample)
    print("Best_colsample_bytree:",Best_colsample_bytree)
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)
    print("Best_n_entimatore:",Best_n_entimatore)
    print("Best_ac:",maxXG)







    if(ac > maxXG):
      maxXG = ac
      Best_gamma = tempGamma
      Best_subsample = subsample
      Best_colsample_bytree = colsample_bytree
      Best_max_depth = max_depth
      Best_min_child_weight = min_child_weight
      Best_n_entimatore = n_entimatore


      print("Best_params:")
      print("Best_gamma:",Best_gamma)
      print("Best_subsample:",Best_subsample)
      print("Best_colsample_bytree:",Best_colsample_bytree)
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)
      print("Best_n_entimatore:",Best_n_entimatore)
      print("Best_ac:",maxXG)








print("Best_params:")
print("Best_gamma:",Best_gamma)
print("Best_subsample:",Best_subsample)
print("Best_colsample_bytree:",Best_colsample_bytree)
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)
print("Best_n_entimatore:",Best_n_entimatore)
print("Best_ac:",maxXG)

Accuracy score :  0.6092529541357901
params:
gamma: 1
subsample: 0.8
colsample_bytree: 0.8
max_depth: 10
min_child_weight: 1
n_entimatore: 550

Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 0.8
Best_max_depth: 0
Best_min_child_weight: 0
Best_n_entimatore: 550
Best_ac: 0
Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 0.8
Best_max_depth: 10
Best_min_child_weight: 1
Best_n_entimatore: 550
Best_ac: 0.6092529541357901
Accuracy score :  0.9086721409973963
params:
gamma: 1
subsample: 0.8
colsample_bytree: 0.8
max_depth: 10
min_child_weight: 2
n_entimatore: 550

Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 0.8
Best_max_depth: 10
Best_min_child_weight: 1
Best_n_entimatore: 550
Best_ac: 0.6092529541357901
Best_params:
Best_gamma: 1
Best_subsample: 0.8
Best_colsample_bytree: 0.8
Best_max_depth: 10
Best_min_child_weight: 2
Best_n_entimatore: 550
Best_ac: 0.9086721409973963


## Light

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_min_child_weight = 0


for numLeaves in range(10,110):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,num_leaves = numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",numLeaves)


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_numLeaves = numLeaves

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_ac:",maxXG)










  

Accuracy score :  0.9105748047266172
Accuracy score :  0.9105748047266172
params:
numLeaves: 10

Best_params:
Best_numLeaves: 0
Best_ac: 0
Best_params:
Best_numLeaves: 10
Best_ac: 0.9105748047266172
Accuracy score :  0.9122771880632886
Accuracy score :  0.9122771880632886
params:
numLeaves: 11

Best_params:
Best_numLeaves: 10
Best_ac: 0.9105748047266172
Best_params:
Best_numLeaves: 11
Best_ac: 0.9122771880632886
Accuracy score :  0.9130783096334869
Accuracy score :  0.9130783096334869
params:
numLeaves: 12

Best_params:
Best_numLeaves: 11
Best_ac: 0.9122771880632886
Best_params:
Best_numLeaves: 12
Best_ac: 0.9130783096334869
Accuracy score :  0.9102743841377929
Accuracy score :  0.9102743841377929
params:
numLeaves: 13

Best_params:
Best_numLeaves: 12
Best_ac: 0.9130783096334869
Accuracy score :  0.9156819547366313
Accuracy score :  0.9156819547366313
params:
numLeaves: 14

Best_params:
Best_numLeaves: 12
Best_ac: 0.9130783096334869
Best_params:
Best_numLeaves: 14
Best_ac: 0.9156819547

## Best_min_child_samples

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 63
Best_min_child_samples = 0
skipper = False
skipper = True

for min_child_samples in range(0,1250,250):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,min_child_samples = min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",min_child_samples)


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_min_child_samples = min_child_samples

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_ac:",maxXG) 

Accuracy score :  0.9147806929701582
Accuracy score :  0.9147806929701582
params:
numLeaves: 63
min_child_samples: 0

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 0
Best_ac: 0
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 0
Best_ac: 0.9147806929701582
Accuracy score :  0.9173843380733027
Accuracy score :  0.9173843380733027
params:
numLeaves: 63
min_child_samples: 250

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 0
Best_ac: 0.9147806929701582
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_ac: 0.9173843380733027
Accuracy score :  0.9156819547366313
Accuracy score :  0.9156819547366313
params:
numLeaves: 63
min_child_samples: 500

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_ac: 0.9173843380733027
Accuracy score :  0.9158822351291809
Accuracy score :  0.9158822351291809
params:
numLeaves: 63
min_child_samples: 750

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 250
Best_ac: 0.9173843380733027
Accuracy score : 

## max_bin

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 63
Best_min_child_samples = 1000
Best_max_bin = 0
skipper = False
skipper = True

for max_bin in range(3,21,3):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,max_bin=max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",min_child_samples)
    print("max_bin:",max_bin)
    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_max_bin = max_bin

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_ac:",maxXG) 

Accuracy score :  0.8991588223512919
Accuracy score :  0.8991588223512919
params:
numLeaves: 63
min_child_samples: 1000
max_bin: 3

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 0
Best_ac: 0
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 3
Best_ac: 0.8991588223512919
Accuracy score :  0.9127778890446625
Accuracy score :  0.9127778890446625
params:
numLeaves: 63
min_child_samples: 1000
max_bin: 6

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 3
Best_ac: 0.8991588223512919
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 6
Best_ac: 0.9127778890446625
Accuracy score :  0.9119767674744642
Accuracy score :  0.9119767674744642
params:
numLeaves: 63
min_child_samples: 1000
max_bin: 9

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 6
Best_ac: 0.9127778890446625
Accuracy score :  0.9130783096334869
Accuracy score :  0.9130783096334869
params:
numLeaves: 63


## Best_bagging_fraction

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 63
Best_min_child_samples = 1000
Best_max_bin = 15
Best_bagging_fraction = 0

skipper = False
skipper = True

bagging_fractionArr = [0.6,0.8,1]


for bagging_fraction in bagging_fractionArr:

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,bagging_fraction=bagging_fraction,max_bin=Best_max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",Best_min_child_samples)
    print("bagging_fraction:",bagging_fraction)
    print("max_bin:",Best_max_bin)
    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_bagging_fraction:",Best_bagging_fraction)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_bagging_fraction = bagging_fraction

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_bagging_fraction:",Best_bagging_fraction)
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_bagging_fraction:",Best_bagging_fraction)
print("Best_ac:",maxXG) 

Accuracy score :  0.9144802723813339
Accuracy score :  0.9144802723813339
params:
numLeaves: 63
min_child_samples: 1000
bagging_fraction: 0.6
max_bin: 15

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0
Best_ac: 0
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0.6
Best_ac: 0.9144802723813339
Accuracy score :  0.9144802723813339
Accuracy score :  0.9144802723813339
params:
numLeaves: 63
min_child_samples: 1000
bagging_fraction: 0.8
max_bin: 15

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0.6
Best_ac: 0.9144802723813339
Accuracy score :  0.9144802723813339
Accuracy score :  0.9144802723813339
params:
numLeaves: 63
min_child_samples: 1000
bagging_fraction: 1
max_bin: 15

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0.6
Best_ac: 0.9144802723813339
Best_params:
Best_numLea

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 63
Best_min_child_samples = 1000
Best_max_bin = 15
Best_bagging_fraction = 0.6
Best_max_depth = 0



skipper = False
skipper = True


for max_depth in range(2,17):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,max_depth=max_depth,bagging_fraction=Best_bagging_fraction,max_bin=Best_max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",Best_min_child_samples)
    print("bagging_fraction:",Best_bagging_fraction)
    print("max_depth:",max_depth)
    print("max_bin:",Best_max_bin)
    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_bagging_fraction:",Best_bagging_fraction)
    print("Best_max_depth:",Best_max_depth)
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_max_depth = max_depth

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_bagging_fraction:",Best_bagging_fraction)
      print("Best_max_depth:",Best_max_depth)
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_bagging_fraction:",Best_bagging_fraction)
print("Best_max_depth:",Best_max_depth)
print("Best_ac:",maxXG) 

Accuracy score :  0.8988584017624675
Accuracy score :  0.8988584017624675
params:
numLeaves: 63
min_child_samples: 1000
bagging_fraction: 0.6
max_depth: 2
max_bin: 15

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0.6
Best_max_depth: 0
Best_ac: 0
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0.6
Best_max_depth: 2
Best_ac: 0.8988584017624675
Accuracy score :  0.9059683556979772
Accuracy score :  0.9059683556979772
params:
numLeaves: 63
min_child_samples: 1000
bagging_fraction: 0.6
max_depth: 3
max_bin: 15

Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0.6
Best_max_depth: 2
Best_ac: 0.8988584017624675
Best_params:
Best_numLeaves: 63
Best_min_child_samples: 1000
Best_max_bin: 15
Best_bagging_fraction: 0.6
Best_max_depth: 3
Best_ac: 0.9059683556979772
Accuracy score :  0.9095734027638694
Accuracy score :  0.9095734027638694
par

## min_split

In [ ]:
import lightgbm as lgb

maxXG = 0
Best_numLeaves = 87
Best_min_child_samples = 500
Best_max_bin = 15
Best_max_depth = 16
Best_min_child_weight = 0

skipper = False
skipper = True

for min_child_weight in range(20,50):

  if(skipper):
    model = lgb.LGBMClassifier(learning_rate=learnLGBM_iHolder,min_child_weight=min_child_weight,max_depth=Best_max_depth,max_bin=Best_max_bin,min_child_samples = Best_min_child_samples,num_leaves = Best_numLeaves)
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)
    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("numLeaves:",Best_numLeaves)
    print("min_child_samples:",Best_min_child_samples)
    print("max_bin:",Best_max_bin)
    print("max_depth:",Best_max_depth)    
    print("min_child_weight:",min_child_weight)    


    print()
    print("Best_params:")
    print("Best_numLeaves:",Best_numLeaves)
    print("Best_min_child_samples:",Best_min_child_samples)
    print("Best_max_bin:",Best_max_bin)    
    print("Best_max_depth:",Best_max_depth)
    print("Best_min_child_weight:",Best_min_child_weight)    
    print("Best_ac:",maxXG)






    if(ac > maxXG):
      maxXG = ac
      Best_min_child_weight = min_child_weight

      print("Best_params:")
      print("Best_numLeaves:",Best_numLeaves)
      print("Best_min_child_samples:",Best_min_child_samples)
      print("Best_max_bin:",Best_max_bin)    
      print("Best_max_depth:",Best_max_depth)
      print("Best_min_child_weight:",Best_min_child_weight)    
      print("Best_ac:",maxXG)



print("Best_params:")
print("Best_numLeaves:",Best_numLeaves)
print("Best_min_child_samples:",Best_min_child_samples)
print("Best_max_bin:",Best_max_bin)    
print("Best_max_depth:",Best_max_depth)
print("Best_min_child_weight:",Best_min_child_weight)    
print("Best_ac:",maxXG) 

Accuracy score :  0.9143801321850591
Accuracy score :  0.9143801321850591
params:
numLeaves: 87
min_child_samples: 500
max_bin: 15
max_depth: 16
min_child_weight: 20

Best_params:
Best_numLeaves: 87
Best_min_child_samples: 500
Best_max_bin: 15
Best_max_depth: 16
Best_min_child_weight: 0
Best_ac: 0
Best_params:
Best_numLeaves: 87
Best_min_child_samples: 500
Best_max_bin: 15
Best_max_depth: 16
Best_min_child_weight: 20
Best_ac: 0.9143801321850591
Accuracy score :  0.9159823753254557
Accuracy score :  0.9159823753254557
params:
numLeaves: 87
min_child_samples: 500
max_bin: 15
max_depth: 16
min_child_weight: 21

Best_params:
Best_numLeaves: 87
Best_min_child_samples: 500
Best_max_bin: 15
Best_max_depth: 16
Best_min_child_weight: 20
Best_ac: 0.9143801321850591
Best_params:
Best_numLeaves: 87
Best_min_child_samples: 500
Best_max_bin: 15
Best_max_depth: 16
Best_min_child_weight: 21
Best_ac: 0.9159823753254557
Accuracy score :  0.9158822351291809
Accuracy score :  0.9158822351291809
params:
nu

## CAT

In [ ]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 0


for iterations in range(200,1000,100):

  if(skipper):



    model = CatBoostClassifier(iterations=iterations,learning_rate=learnCAT_iHolder,depth=2)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",iterations)


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_iterations = iterations

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_ac:",maxXG)

Accuracy score :  0.9081714400160225
Accuracy score :  0.9081714400160225
params:
iterations: 200

Best_params:
Best_iterations: 0
Best_ac: 0
Best_params:
Best_iterations: 200
Best_ac: 0.9081714400160225
Accuracy score :  0.9128780292409373
Accuracy score :  0.9128780292409373
params:
iterations: 300

Best_params:
Best_iterations: 200
Best_ac: 0.9081714400160225
Best_params:
Best_iterations: 300
Best_ac: 0.9128780292409373
Accuracy score :  0.9153815341478069
Accuracy score :  0.9153815341478069
params:
iterations: 400

Best_params:
Best_iterations: 300
Best_ac: 0.9128780292409373
Best_params:
Best_iterations: 400
Best_ac: 0.9153815341478069
Accuracy score :  0.9163829361105548
Accuracy score :  0.9163829361105548
params:
iterations: 500

Best_params:
Best_iterations: 400
Best_ac: 0.9153815341478069
Best_params:
Best_iterations: 500
Best_ac: 0.9163829361105548
Accuracy score :  0.9173843380733027
Accuracy score :  0.9173843380733027
params:
iterations: 600

Best_params:
Best_iterations

## Best_l2_leaf_reg

In [ ]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 0

l2_leaf_regArr = [1,3,5,10,100]


for l2_leaf_reg in l2_leaf_regArr:

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,l2_leaf_reg=l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=2)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",iterations)
    print("l2_leaf_reg:",l2_leaf_reg)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_l2_leaf_reg = l2_leaf_reg

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_ac:",maxXG)

Accuracy score :  0.9197877027838974
Accuracy score :  0.9197877027838974
params:
iterations: 900
l2_leaf_reg: 1

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 1
Best_ac: 0.9197877027838974
Accuracy score :  0.9204886841578209
Accuracy score :  0.9204886841578209
params:
iterations: 900
l2_leaf_reg: 3

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 1
Best_ac: 0.9197877027838974
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_ac: 0.9204886841578209
Accuracy score :  0.917684758662127
Accuracy score :  0.917684758662127
params:
iterations: 900
l2_leaf_reg: 5

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_ac: 0.9204886841578209
Accuracy score :  0.916783496895654
Accuracy score :  0.916783496895654
params:
iterations: 900
l2_leaf_reg: 10

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_ac: 0.9204886841578209
Accuracy score :  0.9189865812136991
Accuracy score :  0.918986581

## Depth

In [7]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 3
Best_depth = 0

l2_leaf_regArr = [1,3,5,10,100]


for depth in range(1,11):

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,l2_leaf_reg=Best_l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=depth)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",Best_iterations)
    print("l2_leaf_reg:",Best_l2_leaf_reg)
    print("depth:",depth)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_depth:",Best_depth)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_depth = depth

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_depth:",Best_depth)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_depth:",Best_depth)
print("Best_ac:",maxXG)

Accuracy score :  0.9125776086521129
Accuracy score :  0.9125776086521129
params:
iterations: 1000
l2_leaf_reg: 3
depth: 1

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 1
Best_ac: 0.9125776086521129
Accuracy score :  0.9204886841578209
Accuracy score :  0.9204886841578209
params:
iterations: 1000
l2_leaf_reg: 3
depth: 2

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 1
Best_ac: 0.9125776086521129
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 2
Best_ac: 0.9204886841578209
Accuracy score :  0.9204886841578209
Accuracy score :  0.9204886841578209
params:
iterations: 1000
l2_leaf_reg: 3
depth: 3

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 2
Best_ac: 0.9204886841578209
Accuracy score :  0.9224914880833166
Accuracy score :  0.9224914880833166
params:
iterations: 1000
l2_leaf_reg: 3
depth: 4

Best_params:
Best_iterations: 1000
B

## Bodercount

In [5]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 3
Best_depth = 4
Best_border_count = 0

l2_leaf_regArr = [1,3,5,10,100]
border_countArr = [5,10,20,30,50,100,200]



for border_count in border_countArr:

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,border_count=border_count,l2_leaf_reg=Best_l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=Best_depth)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",Best_iterations)
    print("l2_leaf_reg:",Best_l2_leaf_reg)
    print("border_count:",Best_border_count)
    print("depth:",Best_depth)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_depth:",Best_depth)
    print("Best_border_count:",Best_border_count)
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_border_count = border_count

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_depth:",Best_depth)
      print("Best_border_count:",Best_border_count)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_depth:",Best_depth)
print("Best_border_count:",Best_border_count)
print("Best_ac:",maxXG)

Accuracy score :  0.9204886841578209
Accuracy score :  0.9204886841578209
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 0
depth: 4

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 5
Best_ac: 0.9204886841578209
Accuracy score :  0.9206889645503705
Accuracy score :  0.9206889645503705
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 5
depth: 4

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 5
Best_ac: 0.9204886841578209
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 10
Best_ac: 0.9206889645503705
Accuracy score :  0.9233927498497897
Accuracy score :  0.9233927498497897
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 10
depth: 4

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 10
Best_ac: 0.9206889645503705
Best

## otr_border_count 

In [7]:
from catboost import CatBoostClassifier

maxXG = 0
Best_numLeaves = 0
skipper = False
skipper = True

Best_iterations = 1000
Best_l2_leaf_reg = 3
Best_depth = 4
Best_border_count = 50
Best_min_data_in_leaf = 0


l2_leaf_regArr = [1,3,5,10,100]
border_countArr = [5,10,20,30,50,100,200]



for min_data_in_leaf in range(1,100):

  if(skipper):

    model = CatBoostClassifier(iterations=Best_iterations,min_data_in_leaf=min_data_in_leaf,border_count=Best_border_count,l2_leaf_reg=Best_l2_leaf_reg,learning_rate=learnCAT_iHolder,depth=Best_depth)
    model.fit(X_train, Y_train,verbose=False)
    Y_pred = model.predict(X_test)
    ac = accuracy_score(Y_test, Y_pred)

    print("Accuracy score : ", ac)

    print("Accuracy score : ", ac)
    print("params:")
    print("iterations:",Best_iterations)
    print("l2_leaf_reg:",Best_l2_leaf_reg)
    print("border_count:",Best_border_count)
    print("min_data_in_leaf:",min_data_in_leaf)    
    print("depth:",Best_depth)
    


    print()
    print("Best_params:")
    print("Best_iterations:",Best_iterations)
    print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
    print("Best_depth:",Best_depth)
    print("Best_border_count:",Best_border_count)
    print("Best_min_data_in_leaf:",Best_min_data_in_leaf)
    
    print("Best_ac:",maxXG)


    if(ac > maxXG):
      maxXG = ac
      Best_min_data_in_leaf = min_data_in_leaf

      print("Best_params:")
      print("Best_iterations:",Best_iterations)
      print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
      print("Best_depth:",Best_depth)
      print("Best_border_count:",Best_border_count)
      print("Best_min_data_in_leaf:",Best_min_data_in_leaf)
      print("Best_ac:",maxXG)







print("Best_params:")
print("Best_iterations:",Best_iterations)
print("Best_l2_leaf_reg:",Best_l2_leaf_reg)
print("Best_depth:",Best_depth)
print("Best_border_count:",Best_border_count)
print("Best_min_data_in_leaf:",Best_min_data_in_leaf)
print("Best_ac:",maxXG)

Accuracy score :  0.9240937312237132
Accuracy score :  0.9240937312237132
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 50
min_data_in_leaf: 1
depth: 4

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 50
Best_min_data_in_leaf: 0
Best_ac: 0
Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 50
Best_min_data_in_leaf: 1
Best_ac: 0.9240937312237132
Accuracy score :  0.9240937312237132
Accuracy score :  0.9240937312237132
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 50
min_data_in_leaf: 2
depth: 4

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 50
Best_min_data_in_leaf: 1
Best_ac: 0.9240937312237132
Accuracy score :  0.9240937312237132
Accuracy score :  0.9240937312237132
params:
iterations: 1000
l2_leaf_reg: 3
border_count: 50
min_data_in_leaf: 3
depth: 4

Best_params:
Best_iterations: 1000
Best_l2_leaf_reg: 3
Best_depth: 4
Best_border_count: 50
Best_min_da